This noteboook will guide you through the creation of a basic system in Springtronics.

# Imports

In [ ]:
import sys
sys.path.append('../../')

import Springtronics as spr
import matplotlib.pyplot as plt
import numpy as np

# Single oscillator

In [ ]:
system = spr.MechanicalSystem()

Variables for the oscillator

In [ ]:
m = 1
b= 2
k= 6
force = 1

print(f'Damping coefficient: {b/(2*np.sqrt(k*m))}')

Simulation variables

In [ ]:
filelength = 78001
sr = 16000
x = np.linspace(0, filelength/sr, filelength)

In [ ]:
dofName = 'oscillator'
system.degreesOfFreedom[f'{dofName}'] = spr.ParametricVariable(m) #creation of the dof
system.degreesOfFreedom[f'{dofName}'].parameterized = True
system.interactionPotentials[f'{dofName}_K'] =  spr.IntegerPotential(k) #spring
system.interactionPotentials[f'{dofName}_K'].strength.parameterized = True
system.interactionPotentials[f'{dofName}_K'].degreesOfFreedom[dofName] = 2
system.interactionPotentials[f'{dofName}_B'] =  spr.LocalDamping(dofName, b) #damping
system.interactionPotentials[f'{dofName}_B'].strength.parameterized = True


In [ ]:
system.excitationSources['soundData'] = spr.DirectCInjectionSource(force) #constant force
system.interactionPotentials[f'excitation'] = spr.Excitation('oscillator', 'soundData', 1.0)

In [ ]:
#probe
system.probes['oscillator'] = spr.WindowedA2Probe('oscillator',
                                                    startIndex=0,
                                                    endIndex=filelength)

# Define an adjoint source (used to compute the gradient efficiently: https://en.wikipedia.org/wiki/Adjoint_state_method)
system.interactionPotentials['oscillator_probe'] = system.probes['oscillator'].makeAdjointSource()

In [ ]:
env = spr.CPPEnvironment(numSteps = filelength,
                           timeStep = 1.0/sr,
                           numSweepSteps = 1,
                           numThreads=1)
#simulation environment

In [ ]:
traj = env.getTrajectories(system, deleteTemp=False)

In [ ]:
env.getAmplitudes(system)

In [ ]:
plt.plot(x, traj[:,0])
plt.xlabel('Time')
plt.ylabel('Displacement')
plt.plot(x, force/(2*k)*np.ones(len(x)))
plt.title(f'Final amplitude: {round(traj[-1,0], 5)}\nEnergy: {round(traj[-1,2], 5)}')
plt.grid()

In [ ]:
env.getGradients(system)

# Cantilever

We can add a cantilever and an optomechanical coupling to add a non linearity in the displacement

In [ ]:
m_cant = 10
b_cant = 10
k_cant = 5
gamma = 10

In [ ]:
dofName = 'cantilever'
system.degreesOfFreedom[f'{dofName}'] = spr.ParametricVariable(m_cant)
system.degreesOfFreedom[f'{dofName}'].parameterized = True
system.interactionPotentials[f'{dofName}_K'] =  spr.IntegerPotential(k_cant)
system.interactionPotentials[f'{dofName}_K'].degreesOfFreedom[dofName] = 2
system.interactionPotentials[f'{dofName}_K'].strength.parameterized = True

system.interactionPotentials[f'{dofName}_B'] =  spr.LocalDamping(dofName, b_cant)
system.interactionPotentials[f'{dofName}_B'].strength.parameterized = True

In [ ]:
system.interactionPotentials[f'OptoCoup_oscillator_cantilever'] =  spr.IntegerPotential(-gamma)
system.interactionPotentials[f'OptoCoup_oscillator_cantilever'].degreesOfFreedom['oscillator'] = 2
system.interactionPotentials[f'OptoCoup_oscillator_cantilever'].degreesOfFreedom['cantilever'] = 1
system.interactionPotentials[f'OptoCoup_oscillator_cantilever'].strength.parameterized = True
#optomechanical coupling (gamma)

In [ ]:
#probe cantilever
system.probes['cantilever'] = spr.WindowedA2Probe('cantilever',
                                                    startIndex=0,
                                                    endIndex=filelength)

# Define an adjoint source (used to compute the gradient efficiently: https://en.wikipedia.org/wiki/Adjoint_state_method)
system.interactionPotentials['cantilever_probe'] = system.probes['cantilever'].makeAdjointSource()

In [ ]:
%%time
traj = env.getTrajectories(system, initialConditions=np.zeros(2), deleteTemp=False)

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(8, 4))

axs[0].plot(x, traj[:,0])
axs[0].set_xlabel('Time')
axs[0].set_ylabel('Displacement')
axs[0].set_title(f'Oscillator\nFinal amplitude: {round(traj[-1,0], 5)}\nEnergy: {round(traj[-1,-2], 5)}')
axs[0].grid()

axs[1].plot(x, traj[:,1])
axs[1].set_xlabel('Time')
axs[1].set_ylabel('Displacement')
axs[1].set_title(f'Cantilever\nFinal amplitude: {round(traj[-1,0], 5)}\nEnergy: {round(traj[-1,-1], 5)}')
axs[1].grid()

plt.tight_layout()

In [ ]:
env.getGradients(system)

# Duffing

We can add a duffing potential to the cantilever

In [ ]:
duffing  = 1e5 #lambda

In [ ]:
dofName = 'cantilever'
system.interactionPotentials[f'{dofName}_L'] =  spr.IntegerPotential(duffing)
system.interactionPotentials[f'{dofName}_L'].degreesOfFreedom[dofName] = 4
system.interactionPotentials[f'{dofName}_L'].strength.parameterized = True
#duffing potential (lambda)

In [ ]:
env = spr.CPPEnvironment(numSteps = filelength,
                           timeStep = 1.0/sr,
                           numSweepSteps = 1,
                           numThreads=1)

In [ ]:
traj_duffing = env.getTrajectories(system)

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(8, 4))

axs[0].plot(x, traj[:,0], 'b', label='Without duffing')
axs[0].plot(x, traj_duffing[:,0], 'r', label='With duffing')
axs[0].set_xlabel('Time')
axs[0].set_ylabel('Displacement')
axs[0].set_title(f'Oscillator\nFinal amplitude: {round(traj[-1,0], 5)}\nEnergy: {round(traj[-1,-2], 5)}')
axs[0].grid()
axs[0].legend()

axs[1].plot(x, traj[:,1], 'b', label='Without duffing')
axs[1].plot(x, traj_duffing[:,1], 'r', label='With duffing')
axs[1].set_xlabel('Time')
axs[1].set_ylabel('Displacement')
axs[1].set_title(f'Cantilever\nFinal amplitude: {round(traj[-1,0], 5)}\nEnergy: {round(traj[-1,-1], 5)}')
axs[1].grid()
axs[1].legend()
plt.tight_layout()

In [ ]:
gradient = env.getGradients(system, deleteTemp=False)
print(gradient)
# 1, 2, 4, 5


# Excitation with audio file

In [ ]:
filelength = 78001
sr = 256000
x = np.linspace(0, 2*filelength/sr, filelength)

In [ ]:
file = '/home/louvet/Documents/01_data/00_one_to_four/training/soundfile_4'

In [ ]:
system.excitationSources['soundData']  = spr.BinaryFileSource(fileList=[file],
                                                                fileLength=filelength,
                                                                fileDataType='double',
                                                                log2Upsampling=2)
system.interactionPotentials[f'excitation'] = spr.Excitation('oscillator', 'soundData', 1.0)

In [ ]:
""" env = spr.CPPEnvironment(numSteps = filelength,
                           timeStep = 1.0/sr,
                           numSweepSteps = 1,
                           numThreads=1) """

In [ ]:
res = env.getTrajectories(system)

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(8, 4))

axs[0].plot(x, res[:,0])
axs[0].set_xlabel('Time')
axs[0].set_ylabel('Displacement')
axs[0].set_title(f'Oscillator\nFinal amplitude: {round(res[-1,0], 5)}\nEnergy: {round(res[-1,-2], 5)}')
axs[0].grid()

axs[1].plot(x, res[:,1])
axs[1].set_xlabel('Time')
axs[1].set_ylabel('Displacement')
axs[1].set_title(f'Cantilever\nFinal amplitude: {round(res[-1,0], 5)}\nEnergy: {round(res[-1,-1], 5)}')
axs[1].grid()
plt.tight_layout()

In [ ]:
env.getGradients(system)

# Matrices

In [ ]:
import Springtronics.LinearSimulation as ls

linPoint = ls.makeZeroVector(system)
M, B, K = ls.makeLinearizedModel(system, linPoint)
print("M = ", M)
print("B = ", B)
print("K = ", K)

In [ ]:
state_size = 8 #2*(2 dofs + 2 probes)
adjoint_size = state_size + 4 #2 stiffnesses and 2 dampings

In [ ]:
K_dims = [adjoint_size, adjoint_size]
K_values = [1, 1, 1, 1, -2*k/m, -b/m, -2*k_cant/m_cant, -b_cant/m_cant]
K_indexes = [4, 0, 5, 1, 6, 2, 7, 3, 0, 4, 4, 4, 1, 5, 5, 5]
print("K_dims: ", K_dims)
print("K_values: ", K_values)
print("K_indexes: ", K_indexes)

In [ ]:
Gamma_dims = [adjoint_size, adjoint_size, state_size]
Gamma_values = [-2*gamma/m, -2*gamma/m, -1/m, -1/m, -2*gamma/m_cant, -1/m_cant, -1/m_cant, 1, 1]
Gamma_indexes = [0, 4, 1, 1, 4, 0, 8, 4, 0, 9, 4, 4, 0, 5, 0, 10, 5, 1, 11, 5, 5, 0, 6, 0, 1, 7, 1]
print("Gamma_dims: ", Gamma_dims)
print("Gamma_values: ", Gamma_values)
print("Gamma_indexes: ", Gamma_indexes)


In [ ]:
Lambda_dims = [adjoint_size, adjoint_size, state_size, state_size]
Lambda_values = [4*duffing/m_cant]
Lambda_indexes = [1, 5, 1, 1]
print("Lambda_dims: ", Lambda_dims)
print("Lambda_values: ", Lambda_values)
print("Lambda_indexes: ", Lambda_indexes)